dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [117]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

# ML Models 
from xgboost import XGBRegressor

Input

In [139]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']
models = ["Random Forest", "XG Boost"]
Hypertunning = False


In [156]:
# lags used in grid search hyper tunning 
lags_grid = [2,3,6,[1,2,3],[1,2,3,6],[3,6],[1,2,3,6]]
# lags_grid = [2,6]

# parameters used in grid search hyper tunning 
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000, 1500],
    'max_depth': [2, 5, 8, 10, 12, 15]
}


Custom Functions

In [120]:
# Data filter based on start and end date, then split in X and y based on provided columns 
# split(historical data frame contains drivers and actual, start date , end date, X columns in list, y column in string )
def split(data,start_date_key,end_date_key,X_cols,y_col):
    data = data[(data['TimeKey']>=start_date_key)&(data['TimeKey']<=end_date_key)]
    X = data[['key','TimeKey']+X_cols]
    y = data[[y_col]]
    return X, y

In [121]:
# fit the model 
# if Hypertunning is turned on fit will be in grid search else normal fit with default values will return 
def skforecastfit(forecaster,y_train_loc,param_grid,lags_grid):
    results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = y_train_loc,
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = 'mean_squared_error',
              initial_train_size = len(y_train)-1,
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = False,
              show_progress      = True
          )   

In [138]:
# SK Forecast ML Models
# skforecastpredict(models in list, X train data frame, y train data frame, X test data frame ) 
def skforecastpredict(models, X_train, y_train, X_test,Hypertunning,param_grid,lags_grid):
    # Predicted output data frame 
    Output = pd.DataFrame()

    # converting data frame to series, as sk forecast y accept series 
    y_train_loc = y_train[y_train.columns.values[0]]
    
    # from sklearn.preprocessing import FunctionTransformer
    # transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

    # Random Forest ========================================================================================================================= 
    if "Random Forest" in models:
        randomforestforecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [2]
             )
        if Hypertunning is True:
            skforecastfit(randomforestforecaster,y_train_loc,param_grid,lags_grid)
        else:
            randomforestforecaster.fit(y= y_train_loc)
        y_hat_randomforest = randomforestforecaster.predict_interval(steps=3).reset_index(drop=True)
        y_hat_randomforest.columns = ['Random Forest Y_hat', 'Random Forest Lower Bound', 'Random Forest Upper Bound']
        Output = pd.concat([Output, y_hat_randomforest], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    
    #XG Boost ================================================================================================================================
    if "XG Boost" in models:
        xgboostForecaster = ForecasterAutoreg(
                regressor= XGBRegressor(random_state = 213),
                lags= [1,2]
            )      
        if Hypertunning is True:
            skforecastfit(xgboostForecaster,y_train_loc,param_grid,lags_grid)
        else:
            xgboostForecaster.fit(y=y_train_loc)
        y_hat_xgboost = xgboostForecaster.predict_interval(steps=3).reset_index(drop=True)
        y_hat_xgboost.columns = ['XG Boost Y_hat', 'XG Boost Lower Bound', 'XG Boost Upper Bound']
        Output = pd.concat([Output, y_hat_xgboost], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
        
    return Output

Processing Input

In [123]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [124]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [125]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [126]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [127]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [128]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [129]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


In [130]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train and Test Data

In [131]:
X_train, y_train = split(Historical_data, Historic_start_date_key,Historic_end_date_key,drivers,Historical_data_column[0])
X_test,y_test = split(Historical_data, Forecast_start_date_key, Forecast_end_date_key, drivers, Historical_data_column[0])

Prediction

In [157]:
Hypertunning=True
models = ["~Random Forest", "XG Boost"]
rf = skforecastpredict(models,X_train, y_train, X_test, Hypertunning, param_grid, lags_grid)

Number of models compared: 252.


lags grid:   0%|          | 0/7 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/pyt

In [150]:
rf

,XG Boost Y_hat,XG Boost Lower Bound,XG Boost Upper Bound
0,237.048431,237.047928,237.103456
1,293.504913,293.504410,293.554688
2,313.114166,313.113663,313.268951


In [108]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 10 # Placeholder, the value will be overwritten
             )

# Lags used as predictors
lags_grid = [2,3,6,[1,2,3],[1,2,3,6],[3,6],[1,2,3,6]]

lags_grid = [2,3]
# Regressor hyperparameters
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000, 1500],
    'max_depth': [2, 5, 8, 10, 12, 15]
}


# lags_grid = [2, 10, [1, 2, 3, 20]]

# Regressor hyperparameters
# param_grid = {
#     'n_estimators': [50, 100],
#     'max_depth': [5, 10, 15]
# }



In [151]:
rf
yhat = rf.rename(columns={'Random Forest Y_hat':'pred','XG Boost Y_hat':'pred'})

yhat

,pred,XG Boost Lower Bound,XG Boost Upper Bound
0,237.048431,237.047928,237.103456
1,293.504913,293.504410,293.554688
2,313.114166,313.113663,313.268951


In [144]:
yhat = forecaster.predict_interval(3)
yhat.reset_index(inplace=True,drop=True)
yhat

NotFittedError: This Forecaster instance is not fitted yet. Call `fit` with appropriate arguments before using predict.

In [152]:
y_test.reset_index(inplace=True,drop=True)
y_test


,Actual
0,209.0
1,223.0
2,333.0


In [153]:
yhat = pd.merge(yhat[['pred']],y_test,left_index=True,right_index=True,how='outer')
yhat

,pred,Actual
0,237.048431,209.0
1,293.504913,223.0
2,313.114166,333.0


In [154]:
yhat['at'] = 'all'
yhat['diff'] = abs(yhat['pred'] - yhat['Actual'])
yhat = yhat.groupby(['at'],as_index=False)[['diff','Actual']].sum()
yhat

,at,diff,Actual
0,all,118.439178,765.0


In [155]:
print("Accuracy",100-yhat['diff']/yhat['Actual']*100)


Accuracy 0    84.517754
dtype: float64
